In [105]:
import pandas as pd
import numpy as np

In [106]:
path_in = "/Users/timholdsworth/code/scaling-science/Data/100_most_impactful_papers.csv"
def get_data(path_in):
    df = pd.read_csv(path_in)
    return df

In [120]:
# Returns a series of len(df) where each value is the column the data starts in for each row
def get_start_columns(df):
    
    total_years_pub = df.loc[:, 'total_years_pub']
    # Because there are 3 extra column at the end
    start_col_pos = total_years_pub + 3
    # Because we want the last certain number of columns
    start_cols = -start_col_pos
    
    return start_cols

In [108]:
# Iterates through rows of a dataframe and updates scores by multiplying them by exp decay factor
def exp_decay(df, start_col):
    
    for index, row in df.iterrows():
        print(index)
        
        # Get the nondecayed scores 
        start = start_col[index]
        impact_scores = df.iloc[index, start:-3].reset_index(drop=True)
        
        # Generate a series of decay coefficients
        time = np.arange(len(impact_scores))
        decay_list = [np.exp(-t / 25) for t in time]
        decay_series = pd.Series(decay_list)
        
        # Multiply the decay coefficeints by the nondecayed scores
        decayed_score = decay_series.multiply(impact_scores)
        
        # Decayed frame is for one paper and correctly updates with each iteration
        # Turn the decayed_score into a df with column names matching and back to year-indexed series
        decayed_frame = decayed_score.to_frame()
        cols = list(df.columns.values)
        year = int(cols[start])
        year_list = ['' + str(year + t) + '' for t in time]
        year_series = pd.Series(year_list)
        decayed_frame['years'] = year_series
        decayed_frame = decayed_frame.set_index('years').T 
        #decayed_series1 = decayed_frame.squeeze()
        
        # This seems to mis
        df.update(decayed_frame)
        
        #print(impact_scores)
        #print(decayed_frame)
        #print(decayed_series1.index)
        #print(df.columns.values)
        
        #df.iloc[index, start:-3].update(decayed_series1)
        
        # Debugging notes: impact score, index, decayed_frame updates fine with iterations
        # The dataframe only updates after the first iteration
        
        
        # TODO figure out why this works only to update the first row - do I need to reset something 
        # at the top of the method
        # df.update(decayed_frame) works to update values in the first row but misallocates the values
        #df.update(decayed_frame)
        #print(df.iloc[index, -7:-3])
        
        #print('Printing decayed frame')
        #print(decayed_frame)
        #print(type(decayed_series_indexed))
        
    
    return df

In [121]:
# Take in a csv with a bunch of data that has year scores and apply exponential decay
def main():
    
    df = get_data(path_in).head(5)
    start_cols = get_start_columns(df)
    df_decayed = exp_decay(df, start_cols)
    return df_decayed

df_decayed = main()
#df_decayed

0
1
2
3
4


In [ ]:
path_out = '/Users/timholdsworth/code/scaling-science/Data/100_most_impactful_papers_decayed.csv'
def write_to_csv(df):
    df.to_csv(path_out, index=False)

In [ ]:
write_to_csv(df_decayed)

In [ ]:
df000 = get_data(path_in).head(10)
cols = list(df.columns.values)
year = cols[-10]
ind = int(year)
ind

In [99]:
# Play with the df.update until it works
df0 = pd.DataFrame({'A': [1, 2, 3], 'B': [400, 500, 600]})
df1 = pd.DataFrame({'A': [4, 5, 6]})
df0.update(df1)

,A,B
0,4,400
1,5,500
2,6,600


In [147]:
for index, row in df0.iterrows():
    print(index)
    df00 = pd.DataFrame({'A': [index+8, index+9, index+10]})
    df0.update(df00)
    #df00 = df1.squeeze()
    #df0.iloc[index, :].update(df00)
#new_df = pd.DataFrame({'B': [4, 5, 6],'C': [7, 8, 9]})
#df.update(new_df)
df0
df0.T

0
1
2


,0,1,2
A,10,11,12
B,400,500,600


In [178]:
# Method to calculate decay_scores for a given paper, returning the scores as a series
def calc_decay_scores(df, start_col, index):
         
    # Get the nondecayed scores 
    start = start_col[index]
    #impact_scores = df.iloc[index, start:-3].reset_index(drop=True)
    impact_scores = df.iloc[start:-3, index].reset_index(drop=True)
    #print(impact_scores)
    
    # Generate a series of decay coefficients
    time = np.arange(len(impact_scores))
    decay_list = [np.exp(-t / 25) for t in time]
    decay_series = pd.Series(decay_list)
    #print(decay_series)

    # Multiply the decay coefficeints by the nondecayed scores
    decay_score = decay_series.multiply(impact_scores)
    return decay_score
    

In [235]:
# Method to update the dataframe with the impact scores
def update_df_with_decay_scores(df, start_cols):
    
    # Take in a series of decay scores for a given paper
    # Add correct years as index and turn to a df
    # Update main df with scores
    
    # Set the index to paper title and transpose main df 
    df = df.set_index('title')
    df = df.transpose()
    
    # For all papers, where each column represents a paper
    for column in df:
        
        # Calculate the decay scores for each row
        decay_score = calc_decay_scores(df, start_cols, df.columns.get_loc(column))
        
        # Turn the decayed_score into a df with column names matching and back to year-indexed series
        decay_frame = decay_score.to_frame()
        
        # Get the value of the column title from the dataframe itself - which is the column title
        decay_frame.columns = [list(df.columns.values)[df.columns.get_loc(column)]]
        
        # Decay frame successfully adds column title from dataframe colum - TODO check for off by one
        #print(decay_frame.columns.values)
        
        # TODO: Bug is that all start values are -54 -> makes all years be 1900 
        # Where does index = 2 come from? Need to set index =
        
        # Build an index of years for the decay_frame
        time = np.arange(len(decay_score))
        start = start_cols[index]
        print(index)
        cols = list(df.index.values)
        year = int(cols[start])
        year_list = ['' + str(year + t) + '' for t in time]
        year_series = pd.Series(year_list)
        decay_frame['years'] = year_series
        decay_frame = decay_frame.set_index('years') 
        
        #print(decay_frame)
        
        # Update the values
        df.update(decay_frame)
       
    return df
    
        
#update_df_with_decay_scores(df, start_cols)  
    

        

In [236]:
def main1():
    
    df = get_data(path_in).head(5)
    start_cols = get_start_columns(df)
    df1 = update_df_with_decay_scores(df, start_cols)  
    return df1

df_dec = main1()
df_dec

2
2
2
2
2


title,Zur Frage des Einflusses der Milz auf den Eisenstoffwechsel,Zur Pathogenese des Sogenannten „Röntgenkaters“,Substituirte Stickstoffbromide und ihre Beziehung zur Bromsubstitution in Aniliden und Anilinen,Über die Acetylierung von löslicher Stärke,The action of young's glycotropic factor of the anterior pituitary gland
1900,0.278000,0.278000,0.150000,0.150000,0.150000
1901,0.267099,0.267099,0.267099,0.144118,0.267099
1902,0.256626,0.256626,0.256626,0.138467,0.256626
1903,0.246564,0.246564,0.246564,0.133038,0.246564
1904,0.236896,0.236896,0.236896,0.127822,0.236896
1905,0.227607,0.227607,0.227607,0.122810,0.227607
1906,0.218683,0.218683,0.218683,0.117994,0.218683
1907,0.210108,0.210108,0.210108,0.113368,0.210108
1908,0.201869,0.201869,0.201869,0.108922,0.201869
1909,0.193954,0.193954,0.193954,0.104651,0.193954
